# 1. Load data

In [456]:
import pandas as pd
import sqlite3

path = "/Users/pavel/PycharmProjects/stock-market-app/src/financial_data.db"

sqlFinancials  = """select *
from balance_sheet bs
left join pnl p
on bs.ticker = p.ticker
and bs.type = p.type
and bs.fiscalDateEnding = p.fiscalDateEnding
left join cash_flow cf
on bs.ticker = cf.ticker
and bs.type = cf.type
and bs.fiscalDateEnding = cf.fiscalDateEnding
"""
conn = sqlite3.connect(path)
financials_db = pd.read_sql(sqlFinancials, conn)
conn.close()

financials_db = financials_db.loc[:, ~financials_db.columns.duplicated()]

sqlCandles = "select * from candles"
conn = sqlite3.connect(path)
candles_db = pd.read_sql(sqlCandles, conn)
conn.close()

# Counting forward looking 90-days average
candles = candles_db[['date_time','ticker', 'close']].copy()
candles = candles.sort_values(['ticker', 'date_time'])
candles['avg_90d'] = (
    candles.groupby('ticker')['close']
    .apply(lambda x: x.shift(-90).rolling(90).mean())
    .reset_index(level=0, drop=True)
)


# Merge financials and stock prices data
candles['date_time'] = pd.to_datetime(candles['date_time'])

financials_db['fiscalDateEnding'] = pd.to_datetime(financials_db['fiscalDateEnding'])

candles = candles.sort_values(by='date_time')
financials_db = financials_db.sort_values(by='fiscalDateEnding')

financials_data = pd.merge_asof(
    financials_db,
    candles,
    left_on='fiscalDateEnding',
    right_on='date_time',
    by='ticker',
    direction='nearest'  # use nearest values
)

financials = financials_data[financials_data['type'] == 'quarterly'].copy()
financials['dividendPayout'] = financials['dividendPayout'].fillna(0)

# 2. Generate features

In [458]:
# Create features:
def calculate_features(financials):
    # - ebitda ltm YoY growth: ebitda_ltm_t / ebitda_ltm_t-1
    financials['ebitda_ltm'] = financials['ebitda'].rolling(4).sum()
    financials['ebitda_ltm_g'] = financials['ebitda_ltm'] / financials['ebitda'].shift(1) * 100
    
    financials['ebitda_ltm_g_avg'] = financials['ebitda_ltm_g'] / financials['ebitda'].rolling(4).sum()
    

    # - revenue growth: revenue_ltm_t / revenue_ltm_t-4
    financials['revenue_ltm'] = financials['totalRevenue'].rolling(4).sum()
    financials['revenue_g'] = financials['revenue_ltm'] / financials['revenue_ltm'].shift(4)
    
    # - net_income growth: net_income_t / net_income_t-1
    financials['net_income_ltm'] = financials['netIncome'].rolling(4).sum()
    financials['net_income_g'] = financials['net_income_ltm'] / financials['net_income_ltm'].shift(4)

    # - price growth
    financials['avg_90d_g'] = financials['avg_90d'] / financials['avg_90d'].shift(1)
    
    # - dividend payout ratio
    financials['dividend_payout_ltm'] = financials['dividendPayout'].rolling(4).sum()
    financials['div_payout_ratio'] = financials['dividend_payout_ltm'] / financials['net_income_ltm'].shift(2)
    
    financials['div_payout_avg'] = financials['div_payout_ratio'].rolling(window=4).mean()

    # - cash-to-dividends
    financials['cash_to_div'] = financials['cashAndCashEquivalentsAtCarryingValue'] / financials['dividend_payout_ltm']
    
    # - debt-to-ebitda
    financials['debt_ebitda'] = (financials['shortTermDebt'] + financials['longTermDebt']) / financials['ebitda_ltm']

    # - debt-to-equity
    financials['debt_equity'] = (financials['shortTermDebt'] + financials['longTermDebt']) / financials['totalShareholderEquity']
    
    # - cash-to-assets
    financials['cash_to_assets'] = financials['cashAndCashEquivalentsAtCarryingValue']/financials['totalAssets'] * 100
    
    financials['cash_to_assets_avg'] = financials['cash_to_assets'].rolling(window=4).mean()
    
    # - ebitda margin: ebitda / sales
    financials['ebitda_margin'] = financials['ebitda_ltm'] / financials['revenue_ltm'] * 100
    
    financials['ebitda_margin_avg'] = financials['ebitda_margin'].rolling(window=4).mean()

    # - roe : net profit / equity
    financials['roe'] = financials['net_income_ltm'] / financials['totalShareholderEquity'] * 100
    financials['roe_avg'] = financials['roe'].rolling(window=8).mean()

    # - ebitda margin growth: em_t / em_t-1
    financials['ebitda_margin_g'] = financials['ebitda_margin'] / financials['ebitda_margin'].shift(1)
    
    return financials

In [459]:
financials_feat = financials.groupby('ticker', group_keys=False).apply(calculate_features).copy()

In [463]:
financials_late = financials_feat.groupby('ticker', as_index=False).apply(lambda x: x.loc[x['fiscalDateEnding'].idxmax()]).copy()

# 3. Exploration analysis of dependencies

In [466]:
import matplotlib.pyplot as plt

def create_plots(data, target, features):
    for feature in features:
        plt.scatter(data[feature], data[target])
        plt.xlabel(feature)
        plt.ylabel(target)
        plt.show()

In [468]:
# define target and features
target = 'div_payout_avg'
features = ['ebitda_margin_avg', 'roe_avg', 'cash_to_assets_avg', 'debt_ebitda', 'ebitda_ltm_g_avg']

In [ ]:
create_plots(financials_late, target, features)

In [472]:
financials_late[financials_late['div_payout_avg'] < 0]['ticker']

9       BP
26    INTC
37      MU
53       T
Name: ticker, dtype: object

In [474]:
# removing outliers
final_columns = ['ticker', 'fiscalDateEnding', target] + features

financials_clean = financials_late[
    (financials_late['div_payout_avg'] > 0) & 
    (financials_late['div_payout_avg'] < 1.1) &
    (financials_late['roe_avg'] < 2000) &
    (financials_late['roe_avg'] > -5) & 
    (financials_late['ebitda_ltm_g_avg'] > 0)][final_columns].copy()

In [ ]:
import seaborn as sns 

plt.figure(figsize=(12,4))
sns.heatmap(financials_clean.isnull(),cbar=False,cmap='viridis',yticklabels=False)
plt.title('Missing value in the dataset')

In [478]:
financials_clean = financials_clean.dropna().copy()

In [ ]:
create_plots(financials_clean, target, features)

# 4. Target and features preprocessing

In [483]:
import numpy as np

# Calculate log for target and features
financials_clean['Log_div_payout'] = np.log(financials_late['div_payout_avg'])

target = 'Log_div_payout'

# new_features = []
# for feature in features:

/opt/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [485]:
from sklearn.preprocessing import StandardScaler

# features standartization
scaler = StandardScaler()
financials_clean[features] = scaler.fit_transform(financials_clean[features])
financials_clean.head()

,ticker,fiscalDateEnding,div_payout_avg,ebitda_margin_avg,roe_avg,cash_to_assets_avg,debt_ebitda,ebitda_ltm_g_avg,Log_div_payout
0,AAPL,2024-03-31,0.157966,0.059154,3.974703,0.241780,-0.474476,-1.416715,-1.845376
1,ACN,2024-05-31,0.428729,-1.068525,-0.237590,0.892381,-0.649091,0.852128,-0.846931
3,AMAT,2024-04-28,0.152171,0.017693,0.346517,2.430422,-0.532184,0.605138,-1.882752
4,AMD,2024-03-31,0.058902,-0.533719,-1.064009,-0.495309,-0.565855,1.648311,-2.831888
7,AVGO,2024-05-05,0.587224,1.373367,0.349454,0.741456,0.822233,-0.515408,-0.532349


In [487]:
# train test split
from sklearn.model_selection import train_test_split

X = financials_clean[features]
y = financials_clean[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Linear regression model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)
print("R-квадрат на обучающем наборе: {:.2f}".format(model.score(X_train, y_train))) 
print("R-квадрат на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))